# ΠΤΥΧΙΑΚΗ ΕΡΓΑΣΙΑ

### Μέρος ΙΙ (TabNet)

## Τσάνος Ευάγγελος

# Θέμα : "Ανίχνευση διαδικτυακών Επιθέσεων με χρήση νευρωνικών δικτύων"



---


Περιεχόμενα
--

1.Functions imports and parameters

2.Φόρτωση Δεδομένων για τον TabNet

3.Preprocessing

4.TabNet

5.Αξιολόγηση TabNet

6.Κατέβασμα αρχείων μοντέλου για χρήση απο API


---



# 1.Functions imports and parameters

In [ ]:
!pip install pytorch-tabnet

In [ ]:
# ----------------------------------------------------------
# Βασικά
# ----------------------------------------------------------
import os
import shutil
import pickle
import joblib
import numpy as np
import pandas as pd

# ----------------------------------------------------------
# Οπτικοποίηση
# ----------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# ----------------------------------------------------------
# Google Drive
# ----------------------------------------------------------
from google.colab import drive, files

# ----------------------------------------------------------
# Προεπεξεργασία & Sampling
# ----------------------------------------------------------
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# ----------------------------------------------------------
# Αξιολόγηση
# ----------------------------------------------------------
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)

# ----------------------------------------------------------
# PyTorch & TabNet
# ----------------------------------------------------------
import torch
from pytorch_tabnet.tab_model import TabNetClassifier


In [ ]:
# Mount Google Drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
# To path που έχουμε κατεβάσει τα αρχεία στο GoogleDrive μας
base_path = '/content/drive/My Drive/UNSW-NB15/'

# Επιλογή του μεγέθους του dataset που θα δουλέψουμε για όλους τους αλγόριθμους
# Σε περίπτωση που θέλουμε να τρέξουμε tuning καλό είναι να μην πάμε με
# ολόκληρο το dataset. Ένα 30% θα εκτελεστεί σε ικανοποιητικούς χρόνους.
# Για απευθειας εκτέλεση με τις παρακάτω τιμές σε περιβάλλον google colab
# και επιλεγμένη hi-ram και T4 cpu θα χρειαστούν περίπου 60 λεπτά.
USE_TUNER = False
FRACTION = 1.0 #100%

In [ ]:

# Function που χρησιμοποιείται για την φόρτωση των αρχείων
def load_unsw_data_from_google_drive(data_path: str, fraction: float = 1.0, encoding: str = 'ISO-8859-1', random_state: int = 42):
    train_file = os.path.join(data_path, 'UNSW_NB15_training-set.csv')
    test_file = os.path.join(data_path, 'UNSW_NB15_testing-set.csv')
    try:
        df_train = pd.read_csv(train_file, encoding=encoding)
        df_test = pd.read_csv(test_file, encoding=encoding)

        if 0 < fraction < 1.0:
            df_train = df_train.sample(frac=fraction, random_state=random_state).reset_index(drop=True)
            df_test = df_test.sample(frac=fraction, random_state=random_state).reset_index(drop=True)

        print("Training set loaded. Shape:", df_train.shape)
        print("Testing set loaded. Shape:", df_test.shape)
        return df_train, df_test
    except FileNotFoundError:
        print("Τα αρχεία training ή testing δεν βρέθηκαν. Έλεγξε τη διαδρομή:", data_path)
    except Exception as e:
        print(f"Σφάλμα κατά τη φόρτωση: {e}")
    return None, None

# 2. Φόρτωση Δεδομένων για τον TabNet


In [ ]:
# Φόρτωση αρχείων για τα μοντέλα μας
# εδώ παίζει ρόλο η παράμετρος FRACTION
try:
    df_train, df_test = load_unsw_data_from_google_drive(base_path, fraction=FRACTION)
except FileNotFoundError:
    print("Τα αρχεία δεν βρέθηκαν. Παρακαλώ ελέγξτε τη διαδρομή.")
except Exception as e:
    print(f"Σφάλμα κατά τη φόρτωση των αρχείων: {e}")


if df_train is not None and df_test is not None:
    print(df_train.head())
    df_train.info()
else:
    print("Δεν φορτώθηκαν δεδομένα.")

# 3. Preprocessing


In [ ]:
# ----------------------------------------------------------
# Καθαρισμός και επιλογή χαρακτηριστικών
# ----------------------------------------------------------
features = df_train.columns.tolist()
features = [col for col in features if col not in ['ï»¿id', 'label', 'attack_cat']]

X_train_raw = df_train[features].copy()
X_test_raw  = df_test[features].copy()

# ----------------------------------------------------------
# Αντικατάσταση σπάνιων κατηγοριών
# ----------------------------------------------------------
threshold = 100
for col in X_train_raw.select_dtypes(include='object').columns:
    value_counts = X_train_raw[col].value_counts()
    rare_values = value_counts[value_counts < threshold].index
    X_train_raw[col] = X_train_raw[col].replace(rare_values, 'rare')
    X_test_raw[col]  = X_test_raw[col].replace(rare_values, 'rare')

# ----------------------------------------------------------
# Αντιγραφή για χρήση με TabNet
# ----------------------------------------------------------
X_train_tabnet = X_train_raw.copy()
X_test_tabnet  = X_test_raw.copy()

categorical_cols_tabnet = X_train_tabnet.select_dtypes(include='object').columns.tolist()
cat_idxs = []
cat_dims = []

# ----------------------------------------------------------
# Label Encoding κατηγορικών χαρακτηριστικών και αποθήκευση
# encoders
# ----------------------------------------------------------
for col in categorical_cols_tabnet:
    le = LabelEncoder()
    all_values = pd.concat([X_train_tabnet[col], X_test_tabnet[col]], axis=0).astype(str)
    le.fit(all_values)
    X_train_tabnet[col] = le.transform(X_train_tabnet[col].astype(str))
    X_test_tabnet[col]  = le.transform(X_test_tabnet[col].astype(str))
    cat_idxs.append(X_train_tabnet.columns.get_loc(col))
    cat_dims.append(len(le.classes_))

    os.makedirs("artifacts", exist_ok=True)
    joblib.dump(le, f"artifacts/{col}_encoder.pkl")

# ----------------------------------------------------------
# Label Encoding για τον στόχο (attack_cat)
# ----------------------------------------------------------
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(df_train['attack_cat'])
y_test_encoded  = label_encoder.transform(df_test['attack_cat'])
joblib.dump(label_encoder, "artifacts/target_encoder.pkl")

# ----------------------------------------------------------
# Train/Validation split
# ----------------------------------------------------------
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_tabnet.values,
    y_train_encoded,
    test_size=0.2,
    stratify=y_train_encoded,
    random_state=42
)

# ----------------------------------------------------------
# Υπολογισμός class weights και sample weights
# ----------------------------------------------------------
classes = np.unique(y_train_split)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_split)
class_weight_dict = dict(zip(classes, class_weights))
sample_weights_tabnet = np.array([class_weight_dict[y] for y in y_train_split])

# ----------------------------------------------------------
# Αποθήκευση metadata και artifacts
# ----------------------------------------------------------
joblib.dump(categorical_cols_tabnet, "artifacts/categorical_columns.pkl")
joblib.dump(X_train_tabnet.columns.tolist(), "artifacts/feature_columns.pkl")
joblib.dump(cat_idxs, "artifacts/cat_idxs.pkl")
joblib.dump(cat_dims, "artifacts/cat_dims.pkl")
joblib.dump(class_weight_dict, "artifacts/class_weight_dict.pkl")
joblib.dump(sample_weights_tabnet, "artifacts/sample_weights_tabnet.npy")

# ----------------------------------------------------------
# Εμφάνιση ελέγχου
# ----------------------------------------------------------
print("TabNet preprocessing ολοκληρώθηκε.")
print("Κατηγορικά:", categorical_cols_tabnet)
print("cat_idxs:", cat_idxs)
print("cat_dims:", cat_dims)
print("class_weights:")
for i, label in enumerate(label_encoder.classes_):
    print(f"  {i}: '{label}' ---> weight: {class_weight_dict[i]:.3f}")


#4.TabNet (Training + Hyperparameter Tuning)




In [ ]:
# ----------------------------------------
# TabNet Training / Hyperparameter Tuning
# ----------------------------------------
best_model = None
best_params = None
best_acc = 0

if USE_TUNER:
    param_grid = [
        {"n_d": 16, "n_a": 16, "lr": 0.01, "n_steps": 5, "gamma": 1.0},
        {"n_d": 32, "n_a": 32, "lr": 0.01, "n_steps": 5, "gamma": 1.0},
        {"n_d": 64, "n_a": 64, "lr": 0.01, "n_steps": 3, "gamma": 0.5},
        {"n_d": 128, "n_a": 128, "lr": 0.01, "n_steps": 3, "gamma": 0.5},
        {"n_d": 32, "n_a": 32, "lr": 0.02, "n_steps": 6, "gamma": 1.3},
        {"n_d": 64, "n_a": 64, "lr": 0.001, "n_steps": 7, "gamma": 1.5},
        {"n_d": 32, "n_a": 16, "lr": 0.005, "n_steps": 8, "gamma": 1.7},
    ]

    for i, params in enumerate(param_grid):
        print(f"\nTrial {i+1} με παραμέτρους: {params}")

        clf = TabNetClassifier(
            cat_idxs=cat_idxs,
            cat_dims=cat_dims,
            cat_emb_dim=8,
            n_d=params["n_d"],
            n_a=params["n_a"],
            n_steps=params["n_steps"],
            gamma=params["gamma"],
            lambda_sparse=0.0001,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=params["lr"]),
            mask_type='sparsemax',
            verbose=1,
            seed=42
        )

        clf.fit(
            X_train=X_train_split,
            y_train=y_train_split,
            eval_set=[(X_train_split, y_train_split), (X_val_split, y_val_split)],
            eval_name=['train','val'],
            eval_metric=['accuracy','logloss'],
            max_epochs=50, #για το tuning αφήνω λίγα
            patience=5,
            batch_size=1024,
            virtual_batch_size=128,
            num_workers=0,
            drop_last=False,
            weights=sample_weights_tabnet
        )

        y_pred_val = clf.predict(X_val_split)
        acc = accuracy_score(y_val_split, y_pred_val)
        print(f"Validation Accuracy: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            best_model = clf
            best_params = params

        history = clf.history
else:
    print("\nSkip tuning - χρήση προκαθορισμένων υπερπαραμέτρων.")
    best_params =  {"n_d": 64, "n_a": 64, "lr": 0.001, "n_steps": 7, "gamma": 1.5}
    #{'n_d': 128, 'n_a': 128, 'lr': 0.01, 'n_steps': 3, 'gamma': 0.5}
     #{"n_d": 64, "n_a": 64, "lr": 0.001, "n_steps": 7, "gamma": 1.5}

    best_model = TabNetClassifier(
        cat_idxs=cat_idxs,
        cat_dims=cat_dims,
        cat_emb_dim=8,
        n_d=best_params["n_d"],
        n_a=best_params["n_a"],
        n_steps=best_params["n_steps"],
        gamma=best_params["gamma"],
        lambda_sparse=0.0001,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=best_params["lr"]),
        mask_type='entmax',
        verbose=1,
        seed=42
    )

    best_model.fit(
        X_train=X_train_split,
        y_train=y_train_split,
        eval_set=[(X_train_split, y_train_split), (X_val_split, y_val_split)],
        eval_name=['train','val'],
        eval_metric=['accuracy','logloss'],
        max_epochs=400,
        patience=20,
        batch_size=1024,
        virtual_batch_size=128,
        num_workers=0,
        drop_last=False,
        weights=sample_weights_tabnet
    )

    best_acc = accuracy_score(y_val_split, best_model.predict(X_val_split))
    history = best_model.history




## 5.Αξιολόγηση TabNet

In [ ]:
# --------------------------------------------------------
# Evaluation on Test Set
# --------------------------------------------------------
print(f"\nΚαλύτερη Validation Accuracy: {best_acc:.4f}")
print("Καλύτερες υπερπαραμέτρους:", best_params)

y_pred_test = best_model.predict(X_test_tabnet.values)


# --------------------------------------------------------
# Confusion Matrix
# --------------------------------------------------------
cm = confusion_matrix(y_test_encoded, y_pred_test)
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - TabNet (Best Trial on Test Set)")
plt.show()

# --------------------------------------------------------
# Αποθήκευση καλύτερου μοντέλου
# --------------------------------------------------------
export_path = "artifacts"
os.makedirs(export_path, exist_ok=True)
best_model.save_model(os.path.join(export_path, "tabnet_best_model"))

# --------------------------------------------------------
# Διαγράμματα εκπαίδευσης (Loss/Accuracy)
# --------------------------------------------------------
plt.figure()
plt.plot(history['loss'], label='Train Loss')
plt.plot(history['val_logloss'], label='Validation Loss', linestyle='--')
plt.title("Loss ανά Εποχή (TabNet)")
plt.xlabel("Εποχές")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

plt.figure()
plt.plot(history['train_accuracy'], label='Train Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy', linestyle='--')
plt.title("Accuracy ανά Εποχή (TabNet)")
plt.xlabel("Εποχές")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()




# --------------------------------------------------------
# Classification Report
# --------------------------------------------------------
print("\nClassification Report στο Test Set:")
print(classification_report(y_test_encoded, y_pred_test, target_names=label_encoder.classes_,digits=4))

print(f"\nMacro F1 Score:      {f1_score(y_test_encoded, y_pred_test, average='macro'):.4f}")
print(f"Weighted F1 Score:   {f1_score(y_test_encoded, y_pred_test, average='weighted'):.4f}")
print(f"Micro F1 Score:      {f1_score(y_test_encoded, y_pred_test, average='micro'):.4f}")
print(f"Macro Precision:     {precision_score(y_test_encoded, y_pred_test, average='macro'):.4f}")
print(f"Macro Recall:        {recall_score(y_test_encoded, y_pred_test, average='macro'):.4f}")

## 6.Κατέβασμα αρχείων μοντέλου για χρήση απο API

In [ ]:
# Δημιουργία zip από τον φάκελο /content/artifacts
shutil.make_archive("artifacts", 'zip', "/content/artifacts")

# katέβασμα τοπικά
files.download("artifacts.zip")